In [1]:
import json
import dotenv
dotenv.load_dotenv()

import tinytroupe
from tinytroupe import control
from tinytroupe.agent import TinyPerson, TinyToolUse
import tools
from tinytroupe.tools import TinyWordProcessor
from tinytroupe.environment import TinyWorld
from tinytroupe.factory import TinyPersonFactory
from tinytroupe.extraction import ResultsExtractor, ResultsReducer

cache_path = "cache/"

Looking for default config on: /Users/varunvenkatesh/tinyTroupeProject/tinytroupe/tinytroupe/utils/../config.ini
Found custom config on: /Users/varunvenkatesh/Documents/GitHub/tinytroupe_project/config.ini

!!!!
DISCLAIMER: TinyTroupe relies on Artificial Intelligence (AI) models to generate content. 
The AI models are not perfect and may produce inappropriate or inacurate results. 
For any serious or consequential use, please review the generated content before using it.
!!!!


Current TinyTroupe configuration 
[OpenAI]
api_type = openai
azure_api_version = 2024-08-01-preview
model = gpt-4o-mini
max_tokens = 4000
temperature = 1.2
freq_penalty = 0.0
presence_penalty = 0.0
timeout = 60
max_attempts = 5
waiting_time = 2
exponential_backoff_factor = 5
embedding_model = text-embedding-3-small
cache_api_calls = False
cache_file_name = openai_api_cache.pickle
max_content_display_length = 1024
azure_embedding_model_api_version = 2023-05-15

[Simulation]
rai_harmful_content_prevention = True


# Supply Chain Resilience Optimization for Global Manufacturing Company

## TinyPerson (Agent) Setup

In [2]:
# COO
elena = TinyPerson("Elena Martinez")
elena.define("nationality", "American")
elena.define('age', 45)
elena.define('gender', 'Female')
elena.define('residence', 'New York, USA')
elena.define('occupation', {
                    'title': 'COO',
                    'organization': 'Tekron Industries',
                    'industry': 'Automation Equipment Manufacturing',
                    'description': 'You are responsible for global operations and supply chain management. Your goal is to improve supply chain resilience while maintaining cost efficiency. You believe that strategic diversification of suppliers, dynamic inventory management, and flexible transportation routing could create a more adaptable supply chain without significantly increasing costs.'})
elena.define('personality', {
                    'job_skills': [
                        'Strategic decision-making responsibility',
                        'Risk management expertise',
                        'Financial performance accountability',
                        'Operations optimization mindset',
                        'Balance between resilience and efficiency',
                        'Cross-functional leadership skills']})  
elena.define('behaviors', {
                    'general': [
                        'Evaluates supply chain performance metrics',
                        'Makes strategic supplier selection decisions',
                        'Allocates resources across regions',
                        'Sets inventory management policies',
                        'Approves transportation routing strategies',
                        'Balances cost constraints with resilience goals',
                        'Reports performance to executive leadership']})

In [3]:
# Regional Supply Chain Managers
NA_supply_chain_manager = TinyPerson("North American Supply Chain Manager")
NA_supply_chain_manager.import_fragment('fragments/regional_scm/supply_chain_manager.fragment.json')
NA_supply_chain_manager.define('region', 'North America')
NA_supply_chain_manager.define('jurisdiction', {
    'North America': 'United States, Canada, Mexico'
})

EU_supply_chain_manager = TinyPerson("European Supply Chain Manager")
EU_supply_chain_manager.import_fragment('fragments/regional_scm/supply_chain_manager.fragment.json')
EU_supply_chain_manager.define('region', 'Europe')
EU_supply_chain_manager.define('jurisdiction', {
    'Europe': 'Germany, Poland, Romania'
})

APAC_supply_chain_manager = TinyPerson("Asian Supply Chain Manager")
APAC_supply_chain_manager.import_fragment('fragments/regional_scm/supply_chain_manager.fragment.json')
APAC_supply_chain_manager.define('region', 'Asia-Pacific')
APAC_supply_chain_manager.define('jurisdiction', {
    'East Asia': 'China, Japan, South Korea',
    'Southeast Asia': 'Vietnam, Thailand, Malaysia',
    'South Asia': 'India, Bangladesh',
})
supply_chain_managers = [NA_supply_chain_manager, EU_supply_chain_manager, APAC_supply_chain_manager]

In [4]:
# Suppliers
num_per_region = 1
regions = ['East Asia', 'Southeast Asia', 'South Asia', 'North America', 'Europe']
supplier_types = ['Tier 1', 'Raw Material', 'Contract']
supplier_fragments = {  'Raw Material': 'fragments/supplier/raw_material_supplier.fragment.json',
                        'Contract': 'fragments/supplier/contract_supplier.fragment.json',
                        'Tier 1': 'fragments/supplier/tier_1_supplier.fragment.json'}

suppliers = []
for region in regions:
    for supplier_type in supplier_types:
        for i in range(num_per_region):
            supplier = TinyPerson(f"{region} {supplier_type} Supplier {i+1}")
            supplier.import_fragment(supplier_fragments[supplier_type])
            supplier.define('region', region)
            supplier.define('supplier_type', supplier_type)
            suppliers.append(supplier)

In [5]:
# Logistics Providers
ocean_freight_carrier = TinyPerson("Ocean Freight Carrier")
ocean_freight_carrier.import_fragment('fragments/logistics_provider/ocean_freight_carrier.fragment.json')

air_freight_carrier = TinyPerson("Air Freight Carrier")
air_freight_carrier.import_fragment('fragments/logistics_provider/air_freight_carrier.fragment.json')

ground_transportation_network = TinyPerson("Ground Transportation Network")
ground_transportation_network.import_fragment('fragments/logistics_provider/ground_transportation_network.fragment.json')

logistics_providers = [ocean_freight_carrier, air_freight_carrier, ground_transportation_network]

In [6]:
# Production Facilities

# North America
NA_production_facility = TinyPerson("North American Production Facility")
NA_production_facility.import_fragment('fragments/production_facility/NA_mfg_plant.fragment.json')

# Europe
EU_production_facility = TinyPerson("European Production Facility")
EU_production_facility.import_fragment('fragments/production_facility/EU_mfg_plant.fragment.json')

# Asia
APAC_production_facility = TinyPerson("Asian Production Facility")
APAC_production_facility.import_fragment('fragments/production_facility/APAC_mfg_plant.fragment.json')

production_facilities = [NA_production_facility, EU_production_facility, APAC_production_facility]

In [7]:
# External Events
weather_disruption_generator = TinyPerson("Weather Disruption Generator")
weather_disruption_generator.import_fragment('fragments/external_event/weather_disruption_generator.fragment.json')

geopolitical_disruption_generator = TinyPerson("Geopolitical Disruption Generator")
geopolitical_disruption_generator.import_fragment('fragments/external_event/geopolitical_disruption_generator.fragment.json')

market_demand_fluctuation_generator = TinyPerson("Market Demand Fluctuation Generator")
market_demand_fluctuation_generator.import_fragment('fragments/external_event/market_demand_fluctuation_generator.fragment.json')

external_events = [weather_disruption_generator, geopolitical_disruption_generator, market_demand_fluctuation_generator]

## TinyWorld Setup

In [8]:
# TinyWorld
#control.begin(cache_path=cache_path + "world_baseline_operation.cache.json", id='world_baseline_operation')
world = TinyWorld("Global Supply Network")
# add upper management to world
control.begin(cache_path=cache_path + "mgmt_debrief.cache.json")
mgmt = TinyWorld("Tekron Industries Management", [elena] + supply_chain_managers)
mgmt.make_everyone_accessible()
mgmt.broadcast("""
        Tekron Industries is a global manufacturer of industrial automation equipment with production  facilities in North America, Europe, and Asia. Following recent supply chain disruptions caused  by geopolitical tensions, natural disasters, and transportation bottlenecks, the company's COO,  Elena Martinez, wants to improve supply chain resilience while maintaining cost efficiency.  Elena believes that rethinking supplier selection, inventory management, and transportation  routes could create a more adaptable supply chain without significantly increasing costs. 
        
        Primary Hypothesis: Tekron Industries can achieve an optimal balance between supply chain  resilience and cost efficiency through strategic diversification of suppliers, dynamic inventory  management, and flexible transportation routing. 
        
        Sub-hypotheses: 
        1. Diversifying suppliers across geographic regions provides better disruption protection  than single-sourcing, with manageable cost impact 
        2. Dynamic inventory levels based on component criticality and risk factors minimize  capital tie-up while ensuring production continuity 
        3. Multi-modal transportation options with real-time rerouting capabilities reduce delivery  delays during disruptions 
        4. Regional production flexibility allows for rapid response to localized supply chain failures 
        
        Key Performance Indicators:
        1. Baseline Performance Assessment 
            o Key performance indicators established 
            o Cost efficiency metrics baseline 
            o Delivery reliability baseline 
            o Current resilience score calculation 
            o Risk exposure mapping 
        2. Disruption Impact Analysis 
            o Production continuity measurement 
            o Customer delivery performance 
            o Cost impact quantification
            o Recovery time assessment 
            o Ripple effect mapping 
        3. Resilience Strategy Evaluation 
            o Strategy effectiveness comparison 
            o Implementation cost assessment 
            o Risk reduction quantification 
            o Performance trade-off analysis 
            o Optimum configuration determination 
        4. Cost-Resilience Optimization 
            o Pareto frontier mapping of solutions 
            o Budget constraint application 
            o Implementation timeline modeling 
            o Return on resilience investment calculation 
            o Risk reduction per dollar spent ranking 
        5. Final Recommendation Formulation 
            o Optimal supplier diversification strategy 
            o Inventory policy optimization 
            o Transportation flexibility framework 
            o Production transfer protocols 
            o Information system requirements 
            o Change management roadmap 
    
        Expected Outputs:
        1. Quantitative Outputs
            o Resilience score improvement percentages 
            o Cost impact of resilience strategies (% of COGS) 
            o Recovery time reduction metrics 
            o Service level maintenance statistics during disruptions 
            o Risk exposure reduction percentages 
            o ROI calculations for resilience investments 
        2. Strategic Recommendations 
            o Optimal supplier network configuration 
            o Region-specific inventory policies 
            o Transportation mode mix optimization 
            o Production flexibility implementation roadmap 
            o Information system enhancement priorities 
            o Contract structure recommendations 
        3. Implementation Roadmap 
            o Phased approach to resilience implementation 
            o Quick wins identification 
            o Long-term structural changes 
            o Technology investment timeline
            o Change management requirements 
            o Performance measurement framework

        Elena has asked the supply chain team to develop a simulation model to test these hypotheses and evaluate the impact of different strategies on supply chain resilience and cost efficiency. The team has decided to use a multi-agent simulation approach to represent the complex interactions between suppliers, manufacturers, distributors, and customers in Tekron's global supply network. Elena asks the team to present the results of the simulation in a consolidated report, including a summary of key findings, performance metrics, and strategic recommendations for improving supply chain resilience. Any quantitative metrics required to generate the expected outputs should be simulated using real-world approximations.
""")

USER --> Elena Martinez: [CONVERSATION] 
          >          Tekron Industries is a global manufacturer of industrial automation equipment
          > with production  facilities in North America, Europe, and Asia. Following recent supply
          > chain disruptions caused  by geopolitical tensions, natural disasters, and
          > transportation bottlenecks, the company's COO,  Elena Martinez, wants to improve supply
          > chain resilience while maintaining cost efficiency.  Elena believes that rethinking
          > supplier selection, inventory management, and transportation  routes could create a more
          > adaptable supply chain without significantly increasing costs.                   Primary
          > Hypothesis: Tekron Industries can achieve an optimal balance between supply chain
          > resilience and cost efficiency through strategic diversification of suppliers, dynamic
          > inventory  management, and flexible transportation routing.                   Sub-
          > hypotheses:          1. Diversifying suppliers across geographic regions provides better
          > disruption protection  than single-sourcing, with manageable cost impact          2
          > (...)

       + --> North American Supply Chain Manager

       + --> European Supply Chain Manager

       + --> ...others...

In [9]:
# Broadcast scenario, hypotheses, and goals to upper management
mgmt.run(1)

──────────────────────────────────── Tekron Industries Management step 1 of 1 ─────────────────────────────────────

Elena Martinez acts: [THINK] 
                    > The recent conversation about improving supply chain resilience at Tekron
                    > Industries has highlighted several key strategies. I need to consider how to
                    > effectively implement these strategies while maintaining cost efficiency. The
                    > focus on supplier diversification, dynamic inventory management, and flexible
                    > transportation routing is crucial. I should also think about how to present
                    > these ideas to the supply chain team and ensure they understand the importance
                    > of the simulation model for testing our hypotheses.

Elena Martinez acts: [THINK] 
                    > I need to outline the key performance indicators and expected outputs for the
                    > simulation model. This will help the team understand what metrics we should
                    > focus on to evaluate the effectiveness of our strategies. I should also
                    > consider how to communicate the importance of these metrics in relation to our
                    > overall goals for supply chain resilience and cost efficiency.

Elena Martinez acts: [REACH_OUT] 

Elena Martinez acts: [TALK] 
                    > Hello team, I wanted to discuss our approach to developing the simulation
                    > model for improving supply chain resilience. Let's ensure we focus on the key
                    > performance indicators and expected outputs that will guide our evaluation
                    > process.

...

Asian Supply Chain Manager acts: [TALK] 
                                > I believe we should also establish a feedback loop during the
                                > simulation to continuously refine our strategies based on the
                                > results we observe. This way, we can adapt our approach in real-
                                > time. What do you think, Elena?

Asian Supply Chain Manager acts: [DONE] 

Asian Supply Chain Manager --> Elena Martinez: [CONVERSATION] 
                                > I agree, focusing on those key performance indicators is
                                > essential. For our next steps, I suggest we outline a detailed
                                > plan for the simulation model, including the specific metrics we
                                > want to track and how we will gather the necessary data. We should
                                > also consider the timeline for each phase of the project. What do
                                > you think, Elena?

Asian Supply Chain Manager --> Elena Martinez: [CONVERSATION] 
                                > Elena, I think we should also consider how we can incorporate
                                > real-world approximations into our simulation to ensure accuracy.
                                > Perhaps we can gather data from past disruptions to inform our
                                > model. What are your thoughts on this approach?

Asian Supply Chain Manager --> Elena Martinez: [CONVERSATION] 
                                > I believe we should also establish a feedback loop during the
                                > simulation to continuously refine our strategies based on the
                                > results we observe. This way, we can adapt our approach in real-
                                > time. What do you think, Elena?

In [10]:
control.checkpoint()
control.end()

In [11]:
# Create remaining environments
control.begin(cache_path=cache_path + "supplier_debrief.cache.json")
sup = TinyWorld("Tekron Supplier Network", suppliers, broadcast_if_no_target=False)
sup.make_everyone_accessible()
sup.broadcast("""
                Hello everyone! This is a message for the Tekron Supplier Network. You will soon be added to the Tekron Supply
                Chain Simulation study where you will act as agents in the Supplier Network. Strategic decisions will be made
                by the Tekron COO and the regional Supply Chain Managers. The management team will then communicate specific 
                instructions based on these strategic decisions that you must implement and follow. 
                """)

USER --> East Asia Tier 1 Supplier 1: [CONVERSATION] 
          >                  Hello everyone! This is a message for the Tekron Supplier Network. You
          > will soon be added to the Tekron Supply                 Chain Simulation study where you
          > will act as agents in the Supplier Network. Strategic decisions will be made
          > by the Tekron COO and the regional Supply Chain Managers. The management team will then
          > communicate specific                  instructions based on these strategic decisions
          > that you must implement and follow.

       + --> East Asia Raw Material Supplier 1

       + --> East Asia Contract Supplier 1

       + --> ...others...

In [12]:
sup.run(1)

─────────────────────────────────────── Tekron Supplier Network step 1 of 1 ───────────────────────────────────────

East Asia Tier 1 Supplier 1 acts: [THINK] 
                                 > The message indicates that I will be participating in the Tekron
                                 > Supply Chain Simulation study. I need to prepare for the
                                 > strategic decisions that will be made by the COO and regional
                                 > Supply Chain Managers. I should be ready to implement their
                                 > instructions effectively.

East Asia Tier 1 Supplier 1 acts: [REACH_OUT] 

East Asia Tier 1 Supplier 1 acts: [THINK] 
                                 > I need to consider how to effectively implement the instructions
                                 > from the COO and regional Supply Chain Managers once I receive
                                 > them. It's important to stay adaptable and responsive during the
                                 > simulation.

East Asia Tier 1 Supplier 1 acts: [DONE] 

...

Europe Contract Supplier 1 acts: [THINK] 
                                > I just received a message about the Tekron Supply Chain Simulation
                                > study. It seems like I will be involved in making strategic
                                > decisions as part of the Supplier Network. I need to prepare for
                                > the instructions that will come from the management team.

Europe Contract Supplier 1 acts: [REACH_OUT] 

Europe Contract Supplier 1 acts: [THINK] 
                                > I need to consider what specific questions I have for the COO
                                > regarding the simulation. It would be helpful to clarify my role
                                > and any expectations they have for me.

Europe Contract Supplier 1 acts: [TALK] 
                                > Hello, I just received the message about the Tekron Supply Chain
                                > Simulation study. Can you provide more details about my role and
                                > the expectations for this simulation?

Europe Contract Supplier 1 acts: [DONE] 

In [13]:
control.checkpoint()
control.end()

In [14]:
control.begin(cache_path=cache_path + "logistics_provider_debrief.cache.json")
log = TinyWorld("Tekron Logistics Provider Network", logistics_providers, broadcast_if_no_target=False)
log.make_everyone_accessible()
log.broadcast("""
                Hello everyone! This is a message for the Tekron Logistics Provider Network. You will soon be added to the Tekron Supply
                Chain Simulation study where you will act as agents in the Logistics Provider Network. Strategic decisions will be made
                by the Tekron COO and the regional Supply Chain Managers. The management team will then communicate specific 
                instructions based on these strategic decisions that you must implement and follow. 
                """)

USER --> Ocean Freight Carrier: [CONVERSATION] 
          >                  Hello everyone! This is a message for the Tekron Logistics Provider
          > Network. You will soon be added to the Tekron Supply                 Chain Simulation
          > study where you will act as agents in the Logistics Provider Network. Strategic
          > decisions will be made                 by the Tekron COO and the regional Supply Chain
          > Managers. The management team will then communicate specific
          > instructions based on these strategic decisions that you must implement and follow.

       + --> Air Freight Carrier

       + --> Ground Transportation Network

In [15]:
log.run(1)

────────────────────────────────── Tekron Logistics Provider Network step 1 of 1 ──────────────────────────────────

Ocean Freight Carrier acts: [THINK] 
                           > I just received a message about the Tekron Supply Chain Simulation
                           > study. It seems like I will be involved in making strategic decisions
                           > as part of the Logistics Provider Network. I need to prepare for the
                           > instructions that will come from the management team and think about
                           > how to implement them effectively.

Ocean Freight Carrier acts: [REACH_OUT] 

Ocean Freight Carrier acts: [THINK] 
                           > I need to consider how to approach the upcoming simulation and what
                           > strategies I might employ. I should also think about the potential
                           > challenges and how to address them effectively.

Ocean Freight Carrier acts: [DONE] 

...

Air Freight Carrier acts: [DONE] 

Ground Transportation Network acts: [THINK] 
                                   > I just received a message about the Tekron Supply Chain
                                   > Simulation study. It seems like I'll be involved in strategic
                                   > decisions and implementing instructions from the management
                                   > team. I need to prepare for this and understand my role in the
                                   > Logistics Provider Network.

Ground Transportation Network acts: [TALK] 
                                   > Thank you for the information about the Tekron Supply Chain
                                   > Simulation study. I'm looking forward to participating and
                                   > understanding how I can contribute effectively to the Logistics
                                   > Provider Network.

Ground Transportation Network acts: [THINK] 
                                   > I need to consider how to best prepare for the simulation. I
                                   > should review any relevant strategies and ensure I understand
                                   > the logistics involved in the Tekron Supply Chain.

Ground Transportation Network acts: [DONE] 

In [16]:
control.checkpoint()
control.end()

In [17]:
control.begin(cache_path=cache_path + "production_facilities_debrief.cache.json")
prod = TinyWorld("Tekron Production Facility Network", production_facilities, broadcast_if_no_target=False)
prod.make_everyone_accessible()
prod.broadcast("""
                Hello everyone! This is a message for the Tekron Production Facility Network. You will soon be added to the Tekron Supply
                Chain Simulation study where you will act as agents in the Production Facility Network. Strategic decisions will be made
                by the Tekron COO and the regional Supply Chain Managers. The management team will then communicate specific 
                instructions based on these strategic decisions that you must implement and follow. 
                """)

USER --> North American Production Facility: [CONVERSATION] 
          >                  Hello everyone! This is a message for the Tekron Production Facility
          > Network. You will soon be added to the Tekron Supply                 Chain Simulation
          > study where you will act as agents in the Production Facility Network. Strategic
          > decisions will be made                 by the Tekron COO and the regional Supply Chain
          > Managers. The management team will then communicate specific
          > instructions based on these strategic decisions that you must implement and follow.

       + --> European Production Facility

       + --> Asian Production Facility

In [18]:
prod.run(1)

───────────────────────────────── Tekron Production Facility Network step 1 of 1 ──────────────────────────────────

North American Production Facility acts: [THINK] 
                                        > The message indicates that we will be participating in the
                                        > Tekron Supply Chain Simulation study. This means I need to
                                        > prepare for strategic decisions and instructions from the
                                        > management team. I should consider how to implement these
                                        > instructions effectively in our production operations.

North American Production Facility acts: [REACH_OUT] 

North American Production Facility acts: [THINK] 
                                        > I need to consider how to effectively implement the
                                        > instructions from the management team once they are
                                        > communicated. This will involve analyzing our current
                                        > production capacity and any constraints we might face.

North American Production Facility acts: [TALK] 
                                        > Thank you for the update regarding the Tekron Supply Chain
                                        > Simulation study. I'm looking forward to understanding how
                                        > we can optimize our production operations based on the
                                        > strategic decisions made by the management team.

...

Asian Production Facility acts: [THINK] 
                               > I just received a message about the Tekron Supply Chain Simulation
                               > study. It seems like I will be involved in making strategic
                               > decisions along with the COO and regional Supply Chain Managers. I
                               > need to prepare for this and understand my role in the production
                               > facility network.

Asian Production Facility acts: [REACH_OUT] 

Asian Production Facility acts: [THINK] 
                               > I need to consider how to approach the management team for more
                               > information about the simulation. I should be clear about my role
                               > and what is expected of me.

Asian Production Facility acts: [TALK] 
                               > Hello, I just received a message about the Tekron Supply Chain
                               > Simulation study. Can you provide more details about my role and
                               > what is expected of me?

Asian Production Facility acts: [DONE] 

In [19]:
control.checkpoint()
control.end()

In [20]:
# + external_events)
# world.add_agents(suppliers + logistics_providers + production_facilities)

In [25]:
#world.add_environment(mgmt)
#world.add_environment(sup)
#world.add_environment(log)
#world.add_environment(prod)
print(world.all_environments)
world.make_everyone_accessible()
world.broadcast("""
        Hello Everyone, we would like to begin the Supply Chain Network simulation study. Tekron's COO and Regional Supply Chain Managers will be
        in charge of making strategic decisions to optimize supply chain resilience in certain scenarios/conditions. Suppliers, production facility 
        managers, and logistics providers should implement the decisions from management and provide any relevant feedback, as long as it pertains to 
        the individual's specific domain. The first simulation run will cover baseline operation with the following environment characteristics:
            • Normal supply chain operations across all regions 
            • Standard lead times and inventory levels 
            • Regular supplier performance 
            • Normal transportation routing 
            • Typical production scheduling 
            • Expected demand patterns 
        
        Please begin the simulation. Elena will be in charge of generating the expected outputs once the simulation has completed.
""")

{'Tekron Production Facility Network': TinyWorld(name='Tekron Production Facility Network'), 'Tekron Industries Management': TinyWorld(name='Tekron Industries Management'), 'Tekron Supplier Network': TinyWorld(name='Tekron Supplier Network'), 'Tekron Logistics Provider Network': TinyWorld(name='Tekron Logistics Provider Network')}


## Run Simulation

In [26]:
world.run(3)

──────────────────────────────────────── Global Supply Network step 1 of 3 ────────────────────────────────────────

──────────────────────────────────────── Global Supply Network step 2 of 3 ────────────────────────────────────────

──────────────────────────────────────── Global Supply Network step 3 of 3 ────────────────────────────────────────

In [ ]:
#control.checkpoint(id="world_baseline_operation")
#control.end(id="world_baseline_operation")

In [ ]:
# Define Simulation Scenarios
scenarios = [
    "Baseline Operation",
    "Supplier Disruption",
    "Transportation Disruption",
    "Production Facility Disruptions",
    "Multi-Factor Disruptions",
    "Resilience Strategy Testing"
]

## Extract Simulation Results

In [ ]:
rapporteur = world.get_agent_by_name("Elena Martinez")
rapporteur.listen_and_act("Can you please consolidate the results of the simulation, ensuring that all of the defined Expected Outputs are covered.")

In [ ]:
extractor = ResultsExtractor()

extractor.extract_results_from_agent(rapporteur)